In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")
import _ct_district_grain_data_prep as _ct_district_data_prep
import branca
import branca.colormap as cm
import _report_utils
import calitp_data_analysis.magics
import geopandas as gpd
import pandas as pd
from great_tables import GT
from IPython.display import HTML, Image, Markdown, display, display_html
from slugify import slugify
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import webmap_utils

In [2]:
import google.auth
credentials, project = google.auth.default()
import gcsfs

fs = gcsfs.GCSFileSystem()

In [3]:
# Comment out and leave this cell right below pandas
# district = "06 - Fresno / Bakersfield"
# district = "09 - Bishop"
district = "07 - Los Angeles / Ventura"

In [4]:
# %%capture_parameters
# district

In [5]:
#DISTRICT_DIGEST_URL = (
#    "https://gtfs-digest--cal-itp-data-analyses.netlify.app/district_"
#  f"{slugify(district_str)}"
# )

In [6]:
# district_str = str(district)

In [7]:
# Extract district from district string when doing an sjoin
# between CT districts & routes
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [8]:
# Files for webmaps
boundary_file = f"district_{district_int}_boundary"
transit_routes_file = f"district_{district_int}_transit_routes"
shn_file = f"district_{district_int}_shn"
transit_shn_file = f"district_{district_int}_transit_routes_shn"

In [9]:
# Load Datasets
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)
operator_list = list(operator_df.analysis_name.unique())

In [10]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.analysis_name.unique())
)

In [11]:
ct_district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [12]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(15, district_int)
)

In [13]:
district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [14]:
gtfs_table_df = _ct_district_data_prep.create_gtfs_stats(operator_df)

In [15]:
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(district_int)

# District {district}

These are district summaries for [GTFS Digest](https://gtfs-digest--cal-itp-data-analyses.netlify.app/). 

In [16]:
#Individual transit operators have their pages at: **[{DISTRICT_DIGEST_URL}]#({DISTRICT_DIGEST_URL})**

In [17]:
district_summary = _report_utils.district_stats(operator_df, "caltrans_district")

In [18]:
summary_table1 = (
    GT(
        district_summary.drop(columns=["arrivals_per_stop", "trips_per_operator"]).pipe(
            _report_utils.transpose_summary_stats, district_col="caltrans_district"
        )
    )
    .fmt_number(columns="value", sep_mark = ",")
    .cols_label(index="")
    .tab_header(title=f"District {district} GTFS summary stats")
)

summary_table2 = (
    GT(
        district_summary[
            ["caltrans_district", "arrivals_per_stop", "trips_per_operator"]
        ].pipe(_report_utils.transpose_summary_stats, district_col="caltrans_district")
    )
    .fmt_number("value", decimals=1, sep_mark = ",")
    .cols_label(index="")
)

In [19]:
display(summary_table1)
display(summary_table2)

GT(_tbl_data=         index      Value
0  # Operators         52
1     # routes        793
2      # trips     32,040
3      # stops     30,292
4   # arrivals  1,475,460, _body=<great_tables._gt_data.Body object at 0x7c0468aadc50>, _boxhead=Boxhead([ColInfo(var='index', type=<ColInfoTypeEnum.default: 1>, column_label='', column_align='left', column_width=None), ColInfo(var='Value', type=<ColInfoTypeEnum.default: 1>, column_label='Value', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x7c04684f9950>, _spanners=Spanners([]), _heading=Heading(title='District 07 - Los Angeles / Ventura GTFS summary stats', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7c0469dcc350>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7c0469dceb10>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value='center'), heading_title_font_size=OptionsInfo(scss=True, category='heading', type='px', value='125%'), heading_title_font_weight=OptionsInfo(scss=True, category='heading', type='value', value='initial'), headin

,Value
Arrivals per Stop,48.71
Trips per Operator,616.15


## Routes within the District

In [20]:
color_map = cm.linear.Spectral_11.scale()

In [21]:
color_map = branca.colormap.LinearColormap(
    colors=color_map.colors[7:], vmin=0, vmax=operator_gdf.Number.max()
)

In [23]:
district_gdf.head(1)

,geometry,color,description
9,"POLYGON ((47998.628 -404632.489, 47993.465 -40...","(58, 25, 79)",geometry for district 7


In [ ]:
district_map = webmap_utils.set_state_export(
    district_gdf,
    subfolder = "caltrans_district_digest/",
    filename=boundary_file,
    map_title="District Map",
    overwrite = True
)

In [ ]:
transit_routes = webmap_utils.set_state_export(
    operator_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_routes_file,
    map_title="Transit Routes",
    cmap=color_map,
    color_col="Number",
    existing_state=district_map,
    overwrite = True
    
)

In [ ]:
webmap_utils.render_spa_link(transit_routes["spa_link"], text="Open Routes for all Operators Map")

In [ ]:
webmap_utils.display_spa_map(transit_routes["spa_link"])

## Transit Routes on the State Highway Network
**Only transit routes that have 15% or more if its length on one or more State Highway Network routes are included**

In [ ]:
color_map2 = cm.linear.RdYlBu_11.scale()
# adjust the number of steps as needed

In [ ]:
color_map2 = branca.colormap.LinearColormap(
    colors=color_map2.colors[7:], vmin=0, vmax=100
)

In [ ]:
shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network',
    overwrite = True
)

In [ ]:
# transit_route_shs_gdf.drop(columns = ["geometry"]).sample()

In [ ]:
transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Percentage of Transit Route on SHN Across All Districts",
    existing_state=shn_map,
    legend_url="https://storage.googleapis.com/calitp-map-tiles/transit_route_pct.svg",
    overwrite = True
    
)

In [ ]:
webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")

In [ ]:
webmap_utils.display_spa_map(transit_shn_map["spa_link"])

In [ ]:
GT(
    transit_route_shs_table.sort_values(
        by=[
            "Analysis Name",
            "Percentage of Transit Route on SHN Across All Districts",
        ],
        ascending=[True, False],
    )
)

## GTFS Stats by Operator

In [ ]:
string_cols = gtfs_table_df.select_dtypes(include="object").columns.tolist()

In [ ]:
gtfs_table = (
    GT(gtfs_table_df.sort_values("# Trips", ascending=False))
    .fmt_integer(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Operator Service Miles", "Avg Arrivals per Stop"]
            and (c not in string_cols)
        ]
    )
    .fmt_number(columns=["Operator Service Miles", "Avg Arrivals per Stop"], decimals=1)
    .data_color(
        columns=["# Trips", "Avg Arrivals per Stop"],
        palette=["#e0ecf6", "#376fa9"],
        na_color="lightgray",
    )
    .tab_header(
        title=f"District {district}",
        subtitle="Daily GTFS schedule statistics by operator",
    )
    .cols_align(
        columns=[
            c
            for c in gtfs_table_df.columns
            if c not in ["Organization", "Transit Operator"]
        ],
        align="center",
    )
)

In [ ]:
gtfs_table = _report_utils.great_table_formatting(gtfs_table)
gtfs_table